# Регрессия

### Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet

Загружаем обработанный набор данных.

In [2]:
machine_df = pd.read_csv('../data/machine_preprocessed.csv')

In [3]:
machine_df.head()

,Vendor_Name,Model_Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132


In [4]:
machine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Vendor_Name  209 non-null    object
 1   Model_Name   209 non-null    object
 2   MYCT         209 non-null    int64 
 3   MMIN         209 non-null    int64 
 4   MMAX         209 non-null    int64 
 5   CACH         209 non-null    int64 
 6   CHMIN        209 non-null    int64 
 7   CHMAX        209 non-null    int64 
 8   PRP          209 non-null    int64 
 9   ERP          209 non-null    int64 
dtypes: int64(8), object(2)
memory usage: 16.5+ KB


Отделяем ключевой атрибут.

In [5]:
X = machine_df.iloc[:,2:9].values
y = machine_df.iloc[:,-1].values

In [6]:
print(X)

[[  125   256  6000 ...    16   128   198]
 [   29  8000 32000 ...     8    32   269]
 [   29  8000 32000 ...     8    32   220]
 ...
 [  125  2000  8000 ...     2    14    52]
 [  480   512  8000 ...     0     0    67]
 [  480  1000  4000 ...     0     0    45]]


In [7]:
print(y)

[ 199  253  253  253  132  290  381  381  749 1238   23   24   70  117
   15   64   23   29   22  124   35   39   40   45   28   21   28   22
   28   27  102  102   74   74  138  136   23   29   44   30   41   74
   74   74   54   41   18   28   36   38   34   19   72   36   30   56
   42   34   34   34   34   34   19   75  113  157   18   20   28   33
   47   54   20   23   25   52   27   50   18   53   23   30   73   20
   25   28   29   32  175   57  181  181   32   82  171  361  350  220
  113   15   21   35   18   20   20   28   45   18   17   26   28   28
   31   31   42   76   76   26   59   65  101  116   18   20   20   30
   44   44   82   82  128   37   46   46   80   88   88   33   46   29
   53   53   41   86   95  107  117  119  120   48  126  266  270  426
  151  267  603   19   21   26   35   41   47   62   78   80   80  142
  281  190   21   25   67   24   24   64   25   20   29   43   53   19
   22   31   41   47   99   67   81  149  183  275  382   56  182  227
  341 

### Разбиение выборки

Разбиваем исходные данные на обучающую и контрольную выборки. Обучающая выборка составляет 80% от объема исходных данных, а контрольная составляет, соответственно, 20%.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Объявляем *StandardScaler* для нормализации данных.

In [9]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Линейная регрессия

Обучаем регрессор *LinearRegression*.

In [11]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)

Оценим качество модели.

In [12]:
print(r2_score(y_test, y_pred_lr))
print(mean_squared_error(y_test, y_pred_lr))
print(mean_squared_error(y_test, y_pred_lr, squared=False))
print(mean_absolute_error(y_test, y_pred_lr))

0.9128930294943467
578.2204650693473
24.04621519219495
17.388180491355122


Модель отрабатывает достаточно неплохо, но среднеквадратичная ошибка высока.

Подбор гиперпараметров для линейной регрессии не проводится.

### Регрессия дерева решений

In [13]:
# https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
dtr = DecisionTreeRegressor()
dtr.fit(X_train, y_train)
y_pred_dtr = dtr.predict(X_test)

Оценим качество модели.

In [14]:
print(r2_score(y_test, y_pred_dtr))
print(mean_squared_error(y_test, y_pred_dtr))
print(mean_squared_error(y_test, y_pred_dtr, squared=False))
print(mean_absolute_error(y_test, y_pred_dtr))

0.9373023029720857
416.1904761904762
20.400746951777922
8.523809523809524


Данная модель показала результат лучше, чем линейная регрессия.

Осуществим подбор гиперпараметров, используя *GridSearchCV*.

In [15]:
dtr.get_params().keys()

dict_keys(['ccp_alpha', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'random_state', 'splitter'])

In [16]:
dtr_parameters_grid = {'criterion' : ['mse', 'friedman_mse', 'mae', 'poisson'],
                       'splitter' : ['best', 'random'],
                       'max_depth' : range(1, 20),
                       'min_samples_split' : range(1, 20) 
}

In [18]:
dtr_grid = GridSearchCV(dtr, dtr_parameters_grid, cv=5, n_jobs=-1, verbose=True)
dtr_grid.fit(X_train, y_train)
dtr_grid.best_params_

Fitting 5 folds for each of 2888 candidates, totalling 14440 fits


{'criterion': 'mae',
 'max_depth': 16,
 'min_samples_split': 5,
 'splitter': 'random'}

In [19]:
dtr_g = DecisionTreeRegressor(criterion='mae', max_depth=16, min_samples_split=5, splitter='random')
dtr_g.fit(X_train, y_train)
y_pred_dtr_g = dtr_g.predict(X_test)

In [20]:
print(r2_score(y_test, y_pred_dtr_g))
print(mean_squared_error(y_test, y_pred_dtr_g))
print(mean_squared_error(y_test, y_pred_dtr_g, squared=False))
print(mean_absolute_error(y_test, y_pred_dtr_g))

0.9631543425217821
244.58333333333334
15.63916025026067
7.666666666666667


Результаты после подбора гиперпараметров улучшились.

### LASSO

In [21]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)

Оценим качество модели.

In [22]:
print(r2_score(y_test, y_pred_lasso))
print(mean_squared_error(y_test, y_pred_lasso))
print(mean_squared_error(y_test, y_pred_lasso, squared=False))
print(mean_absolute_error(y_test, y_pred_lasso))

0.920576477695649
527.2173482521351
22.96121399778625
16.26132388457486


Данная модель показала результат незначительно лучше, чем линейная регрессия, но хуже, чем регрессия дерева решений.

Осуществим подбор гиперпараметров, используя *GridSearchCV*.

In [23]:
lasso.get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'])

In [24]:
lasso_parameters_grid = {'alpha' : np.arange(0.01, 1.0, 0.05),
                         'normalize' : [True, False],
                         'max_iter' : [10, 50, 100, 200, 400, 800, 1000, 1500, 2000],
                         'selection' : ['cyclic', 'random'],
                         'warm_start':[True, False]
                         }

In [25]:
lasso_grid = GridSearchCV(lasso, lasso_parameters_grid, cv = 5, n_jobs=-1, verbose=True)
lasso_grid.fit(X_train, y_train)
lasso_grid.best_params_

Fitting 5 folds for each of 1440 candidates, totalling 7200 fits


{'alpha': 0.56,
 'max_iter': 10,
 'normalize': True,
 'selection': 'random',
 'warm_start': False}

In [28]:
lasso_g = Lasso(alpha=0.56, max_iter=10, normalize=True, selection='random', warm_start=False)
lasso_g.fit(X_train,y_train)
y_pred_lasso_g = lasso_g.predict(X_test)

In [29]:
print(r2_score(y_test, y_pred_lasso_g))
print(mean_squared_error(y_test, y_pred_lasso_g))
print(mean_squared_error(y_test, y_pred_lasso_g, squared=False))
print(mean_absolute_error(y_test, y_pred_lasso_g))

0.9289355826956932
471.7291875156144
21.719327510666954
16.83510673118827


Результат после подбора параметров немного улучшился.

### Гребневая регрессия

In [30]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
ridge = Ridge()
ridge.fit(X_train,y_train)
y_pred_ridge = ridge.predict(X_test)

Оценим качество модели.

In [31]:
print(r2_score(y_test, y_pred_ridge))
print(mean_squared_error(y_test, y_pred_ridge))
print(mean_squared_error(y_test, y_pred_ridge, squared=False))
print(mean_absolute_error(y_test, y_pred_ridge))

0.9133402995523682
575.2514638578537
23.98440042731637
17.36800155119095


Данная модель показала результат хуже, чем *Lasso*.

Осуществим подбор гиперпараметров, используя *GridSearchCV*.

In [32]:
ridge.get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'max_iter', 'normalize', 'random_state', 'solver', 'tol'])

In [33]:
ridge_parameters_grid = {'alpha' : np.arange(0.01, 1.0, 0.05),
                         'max_iter' : [10, 50, 100, 200, 400, 800, 1000, 1500, 2000],
                         'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'],
                         'fit_intercept' : [True, False]
                         }

In [34]:
ridge_grid = GridSearchCV(ridge, ridge_parameters_grid, cv = 5, n_jobs=-1, verbose=True)
ridge_grid.fit(X_train, y_train)
ridge_grid.best_params_

Fitting 5 folds for each of 2520 candidates, totalling 12600 fits


{'alpha': 0.7100000000000001,
 'fit_intercept': True,
 'max_iter': 10,
 'solver': 'sag'}

In [35]:
ridge_g = Ridge(alpha=0.71, fit_intercept=True, max_iter=10, solver='sag')
ridge_g.fit(X_train,y_train)
y_pred_ridge_g = ridge_g.predict(X_test)

In [36]:
print(r2_score(y_test, y_pred_ridge_g))
print(mean_squared_error(y_test, y_pred_ridge_g))
print(mean_squared_error(y_test, y_pred_ridge_g, squared=False))
print(mean_absolute_error(y_test, y_pred_ridge_g))

0.9399890597841324
398.35564891578593
19.958848887543237
15.193435546368871


Результаты после подбора гиперпараметров улучшились.

### Elastic Net регрессия

In [126]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
en = ElasticNet()
en.fit(X_train, y_train)
y_pred_en = en.predict(X_test)

Оценим качество модели.

In [127]:
print(r2_score(y_test, y_pred_en))
print(mean_squared_error(y_test, y_pred_en))
print(mean_squared_error(y_test, y_pred_en, squared=False))
print(mean_absolute_error(y_test, y_pred_en))

0.9291245667746671
411.37963331553505
23.64595232101501
17.91687600420865


Данная модель показала неплохой результат.

Осуществим подбор гиперпараметров, используя *GridSearchCV*.

In [128]:
en.get_params().keys()

dict_keys(['alpha', 'copy_X', 'fit_intercept', 'l1_ratio', 'max_iter', 'normalize', 'positive', 'precompute', 'random_state', 'selection', 'tol', 'warm_start'])

In [129]:
en_parameters_grid = {'alpha' : np.arange(0.01, 1.0, 0.05),
                      'l1_ratio' : np.arange(0.1, 1.0, 0.1),
                      'max_iter' : [10, 50 ,100, 200, 400, 800, 1000],
                      'fit_intercept' : [True, False],
                      'normalize' : [True, False],
                      'selection' : ['cyclic', 'random'],
                     }

In [130]:
en_grid = GridSearchCV(en, en_parameters_grid, cv = 5, n_jobs=-1, verbose=True)
en_grid.fit(X_train, y_train)
en_grid.best_params_

Fitting 5 folds for each of 10080 candidates, totalling 50400 fits


{'alpha': 0.11,
 'fit_intercept': True,
 'l1_ratio': 0.5,
 'max_iter': 10,
 'normalize': False,
 'selection': 'random'}

In [131]:
en_g = ElasticNet(alpha=0.11, fit_intercept=True, l1_ratio=0.5, max_iter=10, normalize=False, selection='random')
en_g.fit(X_train, y_train)
y_pred_en_g = en_g.predict(X_test)

In [132]:
print(r2_score(y_test, y_pred_en_g))
print(mean_squared_error(y_test, y_pred_en_g))
print(mean_squared_error(y_test, y_pred_en_g, squared=False))
print(mean_absolute_error(y_test, y_pred_en_g))

0.9364175564030147
399.3099078887352
22.29656071937309
16.62860149836965


Результат немного улучшился.

**Вывод:** наилучший результат после подбора параметров показала регрессия дерева решений.